In [ ]:
# -------------------
# Chat Loop
# -------------------
print("💬 Compatika is ready. Type your message.\n")

while True:
    user_input = input("You: ").strip()
    if user_input == "":
        continue
    reply = model.generate(user_input)
    print("Compatika:", reply)


In [6]:
import torch
import torch.nn as nn
import random

# -------------------
# Device (GPU if available)
# -------------------
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

# -------------------
# Load & Clean Dataset
# -------------------
pairs = []
with open("compatika_dataset.txt", "r", encoding="utf-8") as f:
    lines = f.read().strip().split("\n")

# remove blank lines
lines = [l.strip() for l in lines if l.strip() != ""]

# make pairs
for i in range(0, len(lines), 2):
    if i + 1 < len(lines):
        if lines[i].startswith("USER:") and lines[i+1].startswith("COMPATIKA:"):
            user = lines[i].replace("USER:", "").strip()
            bot = lines[i+1].replace("COMPATIKA:", "").strip()
            pairs.append((user, bot))

print(f"✅ Loaded training examples: {len(pairs)}")
if len(pairs) == 0:
    raise ValueError("❌ Dataset is empty or formatted incorrectly.")

# -------------------
# Build character vocabulary
# -------------------
chars = sorted(list(set("".join(u + b for u,b in pairs))))
stoi = {ch:i+1 for i,ch in enumerate(chars)}
stoi["<pad>"] = 0
itos = {i:ch for ch,i in stoi.items()}

def encode(text):
    return torch.tensor([stoi[c] for c in text if c in stoi], dtype=torch.long).to(device)

def decode(tensor):
    return "".join(itos.get(i, "") for i in tensor.tolist())

# -------------------
# Simple GRU Encoder-Decoder Model
# -------------------
class Encoder(nn.Module):
    def __init__(self, vocab_size, hidden_size):
        super().__init__()
        self.embed = nn.Embedding(vocab_size, hidden_size)
        self.rnn = nn.GRU(hidden_size, hidden_size)

    def forward(self, x):
        emb = self.embed(x).unsqueeze(1)
        _, h = self.rnn(emb)
        return h

class Decoder(nn.Module):
    def __init__(self, vocab_size, hidden_size):
        super().__init__()
        self.embed = nn.Embedding(vocab_size, hidden_size)
        self.rnn = nn.GRU(hidden_size, hidden_size)
        self.fc = nn.Linear(hidden_size, vocab_size)

    def forward(self, h, max_len=60):
        token = torch.tensor([stoi.get(".", 1)], dtype=torch.long).to(device)
        result = []
        for _ in range(max_len):
            emb = self.embed(token).unsqueeze(0).unsqueeze(0)
            out, h = self.rnn(emb, h)
            logits = self.fc(out.squeeze(0).squeeze(0))
            token = torch.argmax(logits, dim=-1)
            result.append(token.item())
        return torch.tensor(result).to(device)

class Compatika(nn.Module):
    def __init__(self, vocab_size, hidden_size=128):
        super().__init__()
        self.encoder = Encoder(vocab_size, hidden_size)
        self.decoder = Decoder(vocab_size, hidden_size)

    def generate(self, text):
        encoded = encode(text)
        h = self.encoder(encoded)
        out = self.decoder(h)
        return decode(out.cpu())

model = Compatika(len(stoi)).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.003)
loss_fn = nn.CrossEntropyLoss()



Using device: cuda
✅ Loaded training examples: 17578


In [8]:
# -------------------
# TRAINING
# -------------------
print("\n🔥 Training Compatika V1-alpha...\n")

for epoch in range(10):   # ← TRAINING LOOP
    random.shuffle(pairs)
    total_loss = 0

    for user, bot in pairs:   # ← DATA LOOP
        enc = encode(user)
        dec = encode(bot)

        if len(enc) < 1 or len(dec) < 2:
            continue

        h = model.encoder(enc)
        inputs = dec[:-1]
        targets = dec[1:].to(device)

        logits_list = []
        for token in inputs:
            token = token.unsqueeze(0)
            emb = model.decoder.embed(token).unsqueeze(0)
            out, h = model.decoder.rnn(emb, h)
            logit = model.decoder.fc(out.squeeze(0).squeeze(0))
            logits_list.append(logit)

        logits = torch.stack(logits_list)
        loss = loss_fn(logits, targets)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    # ✅ THIS MUST BE INSIDE THE FOR EPOCH LOOP — EXACT SPOT
    print(f"Epoch {epoch+1} ✅ Loss: {total_loss/len(pairs):.4f}")




🔥 Training Compatika V1-alpha...



KeyboardInterrupt: 

In [ ]:
# -------------------
# CHAT
# -------------------
print("\n💬 Compatika is ready. Type your message.\n")

while True:
    msg = input("You: ").strip()
    if msg == "":
        continue
    reply = model.generate(msg)
    print("Compatika:", reply)